In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
# imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from talib import RSI
from datetime import datetime as dt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline

idx = pd.IndexSlice

YEAR = 252

### Get data 

In [63]:
raw = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv', parse_dates=True).dropna()
raw.head()

,Date,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,.SPX,.VIX,EUR=,XAU=,GDX,GLD
1,2010-01-04,30.572827,30.950,20.88,133.90,173.08,113.33,1132.99,20.04,1.4411,1120.00,47.71,109.80
2,2010-01-05,30.625684,30.960,20.87,134.69,176.14,113.63,1136.52,19.35,1.4368,1118.65,48.17,109.70
3,2010-01-06,30.138541,30.770,20.80,132.25,174.26,113.71,1137.14,19.16,1.4412,1138.50,49.34,111.51
4,2010-01-07,30.082827,30.452,20.60,130.00,177.67,114.19,1141.69,19.06,1.4318,1131.90,49.10,110.82
5,2010-01-08,30.282827,30.660,20.83,133.52,174.31,114.57,1144.98,18.13,1.4412,1136.10,49.84,111.37


In [64]:
# reshape table 
prices = pd.melt(raw,  id_vars = ['Date'], value_vars = raw.columns,
                 var_name = 'Ticker', 
                 value_name = 'Close') 
# set index by ticker and Data 
prices.rename(columns={'Ticker': 'ticker', 'Date':'date', 'Close' :'close'}, inplace=True) 
prices.set_index(['ticker', 'date'], inplace=True) 
prices.sort_index(level = ['ticker', 'date'], inplace=True)

# select timeframe
start = '2012-01-01'
end = '2017-12-31'
prices = prices.loc[idx[:, start:end], :]
prices.head()
prices.head()

close
ticker date               
.SPX   2012-01-03  1277.06
       2012-01-04  1277.30
       2012-01-05  1281.06
       2012-01-06  1277.81
       2012-01-09  1280.70

### Add signals 

In [65]:
rolling_1m = prices.close.groupby('ticker').rolling(window=21).mean().values
prices['rolling_1m'] = rolling_1m

rolling_rank = prices.rolling_1m.groupby('ticker').rank(ascending=False).values
prices['rolling_rank'] = rolling_rank

prices['rsi'] = prices.close.groupby(level='ticker').apply(RSI)

### Calculate lagged returns

In [66]:
lags = [10] 

In [67]:
for lag in lags: 
    prices[f'returns_{lag}d'] = prices.groupby('ticker').close.pct_change(lag) 
    
for t in lags:
    prices[f'target_{t}d'] = prices.groupby('ticker')[f'returns_{t}d'].shift(-t) 

In [68]:
prices.head(20) 

close  rolling_1m  rolling_rank        rsi  returns_10d  \
ticker date                                                                    
.SPX   2012-01-03  1277.06         NaN           NaN        NaN          NaN   
       2012-01-04  1277.30         NaN           NaN        NaN          NaN   
       2012-01-05  1281.06         NaN           NaN        NaN          NaN   
       2012-01-06  1277.81         NaN           NaN        NaN          NaN   
       2012-01-09  1280.70         NaN           NaN        NaN          NaN   
       2012-01-10  1292.08         NaN           NaN        NaN          NaN   
       2012-01-11  1292.48         NaN           NaN        NaN          NaN   
       2012-01-12  1295.50         NaN           NaN        NaN          NaN   
       2012-01-13  1289.09         NaN           NaN        NaN          NaN   
       2012-01-17  1293.67         NaN           NaN        NaN          NaN   
       2012-01-18  1308.04         NaN           NaN        NaN     0.024259   
       2012-01-19  1314.50         NaN           NaN        NaN     0.029124   
       2012-01-20  1315.38         NaN           NaN        NaN     0.026790   
       2012-01-23  1316.00         NaN           NaN        NaN     0.029887   
       2012-01-24  1314.65         NaN           NaN  81.529945     0.026509   
       2012-01-25  1326.05         NaN           NaN  84.684280     0.026291   
       2012-01-26  1318.43         NaN           NaN  75.413362     0.020078   
       2012-01-27  1316.33         NaN           NaN  73.040181     0.016079   
       2012-01-30  1313.01         NaN           NaN  69.325846     0.018556   
       2012-01-31  1312.41         NaN           NaN  68.646431     0.014486   

                   target_10d  
ticker date                    
.SPX   2012-01-03    0.024259  
       2012-01-04    0.029124  
       2012-01-05    0.026790  
       2012-01-06    0.029887  
       2012-01-09    0.026509  
       2012-01-10    0.026291  
       2012-01-11    0.020078  
       2012-01-12    0.016079  
       2012-01-13    0.018556  
       2012-01-17    0.014486  
       2012-01-18    0.012270  
       2012-01-19    0.008399  
       2012-01-20    0.022442  
       2012-01-23    0.021527  
       2012-01-24    0.024645  
       2012-01-25    0.018031  
       2012-01-26    0.025424  
       2012-01-27    0.019987  
       2012-01-30    0.029520  
       2012-01-31    0.029023

### Linear Regression 

In [69]:
class MultipleTimeSeriesCV:
    ''' 
    Generates tuples of train_idx, test_idx pairs
    Assumes the MultiIndex contains levels 'symbol' and 'date'
    '''
    def __init__(self,
                 n_splits=3,
                 train_period_length=126,
                 test_period_length=21,
                 lookahead=None,
                 shuffle=False):
        self.n_splits = n_splits
        self.lookahead = lookahead
        self.test_length = test_period_length
        self.train_length = train_period_length
        self.shuffle = shuffle

    def split(self, X, y=None, groups=None):
        unique_dates = X.index.get_level_values('date').unique()
        days = sorted(unique_dates, reverse=True)

        split_idx = []
        for i in range(self.n_splits):
            test_end_idx = i * self.test_length
            test_start_idx = test_end_idx + self.test_length
            train_end_idx = test_start_idx + self.lookahead - 1
            train_start_idx = train_end_idx + self.train_length + self.lookahead - 1
            split_idx.append([train_start_idx, train_end_idx,
                              test_start_idx, test_end_idx])

        dates = X.reset_index()[['date']]
        print(dates)
        for train_start, train_end, test_start, test_end in split_idx:
            train_idx = dates[(dates.date > days[train_start])
                              & (dates.date <= days[train_end])].index
            test_idx = dates[(dates.date > days[test_start])
                             & (dates.date <= days[test_end])].index
            if self.shuffle:
                np.random.shuffle(list(train_idx))
            yield train_idx, test_idx

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [70]:
train_period_length = 63
test_period_length = 10
n_splits = int(3 * YEAR/test_period_length)
lookahead =1 

cv = MultipleTimeSeriesCV(n_splits=n_splits,
                          test_period_length=test_period_length,
                          lookahead=lookahead,
                          train_period_length=train_period_length)

In [34]:
i = 0
for train_idx, test_idx in cv.split(X= prices):
    train = prices.iloc[train_idx]
    train_dates = train.index.get_level_values('date')
    test = prices.iloc[test_idx]
    test_dates = test.index.get_level_values('date')
    # make sure they are non-overlapping by making sure lengths match 
    # after dropping duplicates
    df = train.reset_index().append(test.reset_index())
    n = len(df)
    assert n== len(df.drop_duplicates())
    # print(train.groupby(level='symbol').size().value_counts().index[0])
    # this first statement groups by symbol and then makes a frequency table 
    # that maps sizes to their frequencies (.index[0] gives the maximum value)
    # print(train_dates.min().date())
    # this gets minimum date with time included; .date() chops off the date
    print(train.groupby(level='ticker').size().value_counts().index[0],
          train_dates.min(), train_dates.max(),
          test.groupby(level='ticker').size().value_counts().index[0],
          test_dates.min(), test_dates.max())
    i += 1
    if i == 15:
        break

             date
0      2012-01-03
1      2012-01-04
2      2012-01-05
3      2012-01-06
4      2012-01-09
...           ...
18103  2017-12-22
18104  2017-12-26
18105  2017-12-27
18106  2017-12-28
18107  2017-12-29

[18108 rows x 1 columns]
63 2017-09-18 2017-12-14 10 2017-12-15 2017-12-29
63 2017-09-01 2017-11-30 10 2017-12-01 2017-12-14
63 2017-08-18 2017-11-15 10 2017-11-16 2017-11-30
63 2017-08-04 2017-11-01 10 2017-11-02 2017-11-15
63 2017-07-21 2017-10-18 10 2017-10-19 2017-11-01
63 2017-07-07 2017-10-04 10 2017-10-05 2017-10-18
63 2017-06-22 2017-09-20 10 2017-09-21 2017-10-04
63 2017-06-08 2017-09-06 10 2017-09-07 2017-09-20
63 2017-05-24 2017-08-22 10 2017-08-23 2017-09-06
63 2017-05-10 2017-08-08 10 2017-08-09 2017-08-22
63 2017-04-26 2017-07-25 10 2017-07-26 2017-08-08
63 2017-04-11 2017-07-11 10 2017-07-12 2017-07-25
63 2017-03-28 2017-06-26 10 2017-06-27 2017-07-11
63 2017-03-14 2017-06-12 10 2017-06-13 2017-06-26
63 2017-02-28 2017-05-26 10 2017-05-30 2017-06-12


In [40]:
target = f'target_{lookahead}d'
lr_predictions, lr_scores = [], []
lr = LinearRegression()
for i, (train_idx, test_idx) in enumerate(cv.split(prices), 1):
    X_train, y_train, =  prices.iloc[train_idx], y[target].iloc[train_idx]
    X_test, y_test = prices.iloc[test_idx], y[target].iloc[test_idx]
    lr.fit(x=X_train, y=y_train)
    y_pred = lr.predict(X_test)

    preds = y_test.to_frame('actuals').assign(predicted=y_pred)
    preds_by_day = preds.groupby(level='date')
    scores = pd.concat([preds_by_day.apply(lambda x: spearmanr(x.predicted,
                                                               x.actuals)[0] * 100)
                        .to_frame('ic'),
                        preds_by_day.apply(lambda x: np.sqrt(mean_squared_error(y_pred=x.predicted,
                                                                                y_true=x.actuals)))
                        .to_frame('rmse')], axis=1)

    lr_scores.append(scores)
    lr_predictions.append(preds)

lr_scores = pd.concat(lr_scores)
lr_predictions = pd.concat(lr_predictions)

             date
0      2012-01-03
1      2012-01-04
2      2012-01-05
3      2012-01-06
4      2012-01-09
...           ...
18103  2017-12-22
18104  2017-12-26
18105  2017-12-27
18106  2017-12-28
18107  2017-12-29

[18108 rows x 1 columns]


NameError: name 'y' is not defined